<a href="https://colab.research.google.com/github/Dohy-Lee/ML_DL.Basic/blob/main/5_2)Cross_Validation_%26_Greed_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#검증 세트(Validation Set) : 테스트 세트를 사용하지 않고, 훈련 세트를 다시 또 나눔
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()

In [2]:
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target = train_test_split(data,target,test_size=0.2,random_state=42) #sub : 훈련 세트, val : 검증 세트
sub_input,val_input,sub_target,val_target = train_test_split(train_input,train_target, test_size=0.2,random_state = 42)
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [3]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [4]:
#교차 검증(Cross Validation) : 검증 세트를 떼어 내어 평가하는 과정을 반복
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00657868, 0.00666356, 0.00690079, 0.00701475, 0.00644898]), 'score_time': array([0.00058722, 0.00051904, 0.00066233, 0.00070167, 0.00064874]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


fit_time : 모델을 훈련하는 시간 
score_time : 검증하는 시간
test_score : 검증 폴드의 점수

In [5]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [6]:
#cross_validate는 훈련 세트를 섞어 폴드를 나누지 않음. 따라서 train_test_split을 쓰지 않았을 경우,훈련 세트를 섞어 줄 분할기(splitter)를 지정해야함
#일반적으로 KFold : 회귀 모델, StratifiedKFold : 분류모델
#앞서한 코드를 분할기를 사용하여 작성한다면 아래와 같음
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [7]:
splitter = StratifiedKFold(n_splits= 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [8]:
#그리드 서치(Greed Search) : 하이퍼파라미터의 최적의 값을 찾아줌
#GridSearchCV : 하이퍼파리미터 탐색 + 교차 검증
#예제 결정트리에서 min_impurity_decrease 최적값 찾기
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs = -1)
gs.fit(train_input, train_target)
dt = gs.best_estimator_ #best_estimator에는 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련한 모델이 담겨있음
print(dt.score(train_input, train_target))
print(gs.best_params_) #best_params_ : 그리드 서치로 찾은 최적의 매개변수가 저장되어 있음
print(gs.cv_results_['mean_test_score']) #mean_test_score : 각 매개변수에서 수행한 교차 검증의 평균 점수가 저장되어 있음

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [9]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [10]:
params = {'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001),'max_depth' : range(5,20,1), 'min_samples_split' : range(2,100,10)}

In [11]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs = -1)
gs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


In [12]:
from scipy.stats import uniform, randint #uniform, randint : 모두 주어진 범위에서 고르게 값을 뽑아서 '균등 분포에서 샘플링한다'라고 함
rgen = randint(0,10)
rgen.rvs(10)

array([0, 4, 9, 0, 1, 8, 6, 7, 2, 4])

In [13]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 92, 119,  97, 106, 108,  96,  79, 108,  99,  96]))

In [14]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.12255406, 0.81781173, 0.31288622, 0.78138864, 0.25593988,
       0.88062707, 0.85903509, 0.05218288, 0.52978468, 0.09091604])

In [15]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20,50),
          'min_samples_split' : randint(2,25),
          'min_samples_leaf' : randint(1,25),
          }

In [16]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter = 100, n_jobs = -1, random_state = 42)
gs.fit(train_input, train_target)
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [17]:
print(np.max(gs.cv_results_['mean_test_score']))
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.8695428296438884
0.86


In [20]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42, splitter='random'), params, n_iter = 100, n_jobs = -1, random_state = 42)
gs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))
dt = gs.best_estimator_
print(dt.score(test_input, test_target))
#결정 트리의 노드를 무작위로 분할하기에 100번의 반복에서 최적의 매개변수 조합을 찾지 못함. 따라서 이전 결과에 비해 평균 검증 점수와 테스트 세트의 점수가 모두 낮음

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
